In [1]:
!pip install comet_ml -q timm fastai albumentations

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 729.6/729.6 kB 11.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 88.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 71.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 10.9 MB/s eta 0:00:00
ERROR: pip's dependency resol

In [ ]:
import os
import pandas as pd
import numpy as np
from PIL import Image
from tqdm import tqdm

import random
import torch
import timm
import albumentations as alb
from fastai.vision.all import *
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, precision_score, recall_score
import gc
import comet_ml

COMET_API_KEY = "My_comet"
COMET_PROJECT_NAME = "classification-for-landslide-detection-ensamble-pytorch"

DATA_ROOT = "/kaggle/input/slideandseekclasificationlandslidedetectiondataset"
TRAIN_CSV = f"{DATA_ROOT}/Train.csv"
TEST_CSV = f"{DATA_ROOT}/Test.csv"
TRAIN_NPY_PATH = f"{DATA_ROOT}/train_data/train_data"
TEST_NPY_PATH = f"{DATA_ROOT}/test_data/test_data"

class Config:
    n_splits = 3
    seed = 42
    image_size = 196
    model_name = "eva_large_patch14_196.in22k_ft_in22k_in1k"
    batch_size = 16
    epochs = 10 # used only 10 epochs, but using more likely will get better f1
    tta = 4
    num_classes = 2
    IMAGE_COMBINATIONS = [
        ['red', 'green', 'blue'],
        ['nir', 'red', 'green'],
        ['nir', 'green', 'blue'],
        ['nir', 'red', 'blue'],
    ] # Used only 4 combinations and didn't used SAR channels, probably using them will get better f1
    PARENT_SAVE_PATH = "/kaggle/working/landslide_composite_images/"

CFG = Config()

def random_seed(seed_value, use_cuda):
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    random.seed(seed_value)
    if use_cuda:
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

random_seed(CFG.seed, True)

os.makedirs(CFG.PARENT_SAVE_PATH, exist_ok=True)


train_df = pd.read_csv(TRAIN_CSV)
test_df = pd.read_csv(TEST_CSV)

train_df['numpy_path'] = train_df['ID'].apply(lambda x: TRAIN_NPY_PATH + f"/{x}.npy")
test_df['numpy_path'] = test_df['ID'].apply(lambda x: TEST_NPY_PATH + f"/{x}.npy")

label_counts = train_df['label'].value_counts()
print("Distribution of labels in training set:\n", label_counts)

drop_ids = ["ID_Z29R76"]
train_df = train_df[~train_df['ID'].isin(drop_ids)].reset_index(drop=True)

BAND_NAMES = [
    "red", "green", "blue", "nir",
    "desc_vv", "desc_vh", "desc_diff_vv", "desc_diff_vh",
    "asc_vv", "asc_vh", "asc_diff_vv", "asc_diff_vh"
]
def generate_and_save_composite_images(df, combinations, parent_save_path, data_type='train', image_format = 'png'):
    assert image_format in ['png', 'jpg'], "Only png or jpg images are supported"

    for _, row in tqdm(df.iterrows(), total=df.shape[0], desc=f"Generating {data_type} images"):
        tile = np.load(row['numpy_path'])
        assert tile.shape[-1] == len(BAND_NAMES), f"Expected {len(BAND_NAMES)} bands, got {tile.shape[-1]} bands"
        image_id = row['ID']

        for combo in combinations:
            try:
                indices = [BAND_NAMES.index(band) for band in combo]
            except ValueError as e:
                print(f"Invalid band in combo {combo}: {e}")
                continue

            combo_img = tile[:,:,indices]

            combo_img = (combo_img - combo_img.min()) / (combo_img.max() - combo_img.min() + 1e-6)
            combo_img = (combo_img * 255).astype(np.uint8)

            combo_name = "_".join(combo)
            save_dir = os.path.join(parent_save_path, data_type, combo_name)
            os.makedirs(save_dir, exist_ok=True)
            save_path = os.path.join(save_dir, f"{image_id}.{image_format}")

            im = Image.fromarray(combo_img)
            im.save(save_path)

print("Starting composite image generation...")
generate_and_save_composite_images(
     df = train_df,
     combinations = CFG.IMAGE_COMBINATIONS,
     parent_save_path = CFG.PARENT_SAVE_PATH,
     image_format = 'png',
     data_type='train'
)
generate_and_save_composite_images(
     df = test_df,
     combinations = CFG.IMAGE_COMBINATIONS,
     parent_save_path = CFG.PARENT_SAVE_PATH,
     image_format = 'png',
     data_type='test'
)
print("Image generation complete.")

CURRENT_COMBINATION_NAME = 'nir_green_blue'

train_df['image_path'] = train_df['ID'].apply(lambda x: os.path.join(CFG.PARENT_SAVE_PATH, 'train', CURRENT_COMBINATION_NAME, f'{x}.png'))
test_df['image_path'] = test_df['ID'].apply(lambda x: os.path.join(CFG.PARENT_SAVE_PATH, 'test', CURRENT_COMBINATION_NAME, f'{x}.png'))


skf = StratifiedKFold(n_splits=CFG.n_splits, shuffle=True, random_state=CFG.seed)
train_df['fold'] = -1
for fold, (train_idx, val_idx) in enumerate(skf.split(train_df, train_df['label'])):
    train_df.loc[val_idx, 'fold'] = fold
print("\nFold distribution:\n", train_df['fold'].value_counts())

class AlbumentationsTransform(RandTransform):
    split_idx, order = None, 2
    def __init__(self, train_aug, valid_aug): store_attr()

    def before_call(self, b, split_idx):
        self.idx = split_idx

    def encodes(self, img: PILImage):
        if self.idx == 0:
            aug_img = self.train_aug(image=np.array(img))['image']
        else:
            aug_img = self.valid_aug(image=np.array(img))['image']
        return PILImage.create(aug_img)
# Make some augs for train
def get_train_aug():
    return alb.Compose([
        alb.Resize(CFG.image_size, CFG.image_size),
        alb.HorizontalFlip(p=0.5),
        alb.VerticalFlip(p=0.5),
    ], p=1.)

def get_valid_aug():
    return alb.Compose([
        alb.Resize(CFG.image_size, CFG.image_size),
    ], p=1.0)

item_tfms = AlbumentationsTransform(get_train_aug(), get_valid_aug())
batch_tfms = [Normalize.from_stats(*imagenet_stats)]

def get_datablock(df, fold=0, bs = CFG.batch_size):
    return DataBlock(
        blocks = (ImageBlock, CategoryBlock),
        get_x = ColReader('image_path'),
        get_y = ColReader('label'),
        splitter = IndexSplitter(df[df['fold'] == fold].index),
        item_tfms = item_tfms,
        batch_tfms = batch_tfms
    ).dataloaders(df, bs=bs, seed= CFG.seed)

metrics = [accuracy, F1Score(average='binary')]

oof_preds = np.zeros((len(train_df), CFG.num_classes))
all_test_preds = []

for fold in range(CFG.n_splits):
    if COMET_API_KEY != "YOUR_COMET_API_KEY" and COMET_API_KEY != "":
        exp = comet_ml.Experiment(
            api_key=COMET_API_KEY,
            project_name=COMET_PROJECT_NAME,
            auto_output_logging="simple"
        )
        exp.set_name(f"fastai_beitv2_fold_{fold+1}_{CURRENT_COMBINATION_NAME}")

        exp.log_parameters({
            "n_splits": CFG.n_splits,
            "seed": CFG.seed,
            "image_size": CFG.image_size,
            "model_name": CFG.model_name,
            "batch_size": CFG.batch_size,
            "epochs": CFG.epochs,
            "tta_n": CFG.tta,
            "fold_id": fold,
            "image_combination": CURRENT_COMBINATION_NAME
        })
    else:
        print("Comet ML API key not set or is default. Skipping Comet ML logging for this run.")
        exp = None

    print(f"\n{'='*100}")
    print(f"Training on fold : {fold} and validating on fold : {fold}")
    print(f"{'='*100}")

    dls = get_datablock(train_df, fold, CFG.batch_size)

    learn = vision_learner(
        dls,
        CFG.model_name,
        loss_func = CrossEntropyLossFlat(),
        metrics = metrics,
        cbs = [SaveModelCallback(monitor = 'f1_score', comp = np.greater, fname=f'best_model_fold_{fold}_{CURRENT_COMBINATION_NAME}')]
    )

    print("Finding optimal learning rate...")
    lr_min, lr_steep = learn.lr_find(suggest_funcs=(valley, slide))
    print(f"Suggested LR from valley: {lr_min}, from slide: {lr_steep}")
    learn.fine_tune(CFG.epochs, lr_min)
#Logging experiment
    if exp:
        for i, row in enumerate(learn.recorder.values):
            epoch_num = i
            train_loss = row[0]
            val_loss = row[1]
            val_accuracy = row[2]
            val_f1 = row[3]
            current_lr = learn.opt.param_groups[0]['lr']

            exp.log_metrics({
                "train_loss": train_loss,
                "val_loss": val_loss,
                "val_f1": val_f1,
                "val_accuracy": val_accuracy,
                "learning_rate": current_lr
            }, step=epoch_num)

    val_idx = dls.valid.items.index
    val_dl = learn.dls.valid
    oof_val_preds, _ = learn.tta(dl= val_dl, n=CFG.tta)
    oof_preds[val_idx] = oof_val_preds.numpy()

    test_dl = learn.dls.test_dl(test_df)
    preds, _ = learn.tta(dl= test_dl, n=CFG.tta)
    all_test_preds.append(preds.numpy())

    if exp:
        exp.end()

    del learn, dls
    gc.collect()
    torch.cuda.empty_cache()

# Making final averaged preds
print("\n--- Final evaluation ---")
train_df['oof_preds'] = list(oof_preds)

train_df['predicted_label'] = train_df['oof_preds'].apply(
    lambda preds: 1 if preds[1] > 0.5 else 0
)
y_true_oof = train_df['label'].values
y_pred_oof = train_df['predicted_label'].values

final_oof_f1 = f1_score(y_true_oof, y_pred_oof, average='binary', zero_division=0)
print(f"Final F1_Score on OOF predictions: {final_oof_f1:.4f}")

final_test_preds_avg = np.mean(all_test_preds, axis=0)
test_df['label'] = (final_test_preds_avg[:, 1] > 0.5).astype(int)

submission_df = test_df[['ID', 'label']]
submission_output_path = os.path.join(CFG.PARENT_SAVE_PATH, "submission.csv")
submission_df.to_csv(submission_output_path, index=False)
print(f"Submission file saved to: {submission_output_path}")

/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.8' (you have '2.0.4'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


Distribution of labels in training set:
 label
0    5892
1    1255
Name: count, dtype: int64
Starting composite image generation...


Generating test images: 100%|██████████| 5398/5398 [01:07<00:00, 80.39it/s]


Image generation complete.

Fold distribution:
 fold
2    2382
0    2382
1    2382
Name: count, dtype: int64


COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch, sklearn, fastai.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/makar-rybkin/classification-for-landslide-detection-ensamble-pytorch/1c143a3184364c249b8a47cfcdb693b9

COMET INFO: Couldn't find a Git repository in '/kaggle/working' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.


...

epoch,train_loss,valid_loss,accuracy,f1_score,time
0,0.472572,0.302139,0.877414,0.679121,09:04


Better model found at epoch 0 with f1_score value: 0.6791208791208792.


/usr/local/lib/python3.11/dist-packages/fastai/learner.py:53: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(file, map_location=device, **torch_load_kwargs

epoch,train_loss,valid_loss,accuracy,f1_score,time
0,0.237085,0.198115,0.918976,0.735254,12:08
1,0.246559,0.166985,0.927372,0.810515,12:07
2,0.213645,0.170081,0.923174,0.803014,12:08
3,0.174028,0.114600,0.951721,0.855709,12:07
4,0.117723,0.339816,0.890008,0.543554,12:08


Better model found at epoch 0 with f1_score value: 0.7352537722908093.
Better model found at epoch 1 with f1_score value: 0.8105147864184009.
Better model found at epoch 3 with f1_score value: 0.8557089084065245.


In [ ]:
train_df['oof_preds'] = list(oof_preds)

train_df['predicted_label'] = train_df['oof_preds'].apply(
    lambda preds: 1 if preds[1] > 0.5 else 0
)
y_true_oof = train_df['label'].values
y_pred_oof = train_df['predicted_label'].values

final_oof_f1 = f1_score(y_true_oof, y_pred_oof, average='binary', zero_division=0)
print(f"Final F1_Score on OOF predictions: {final_oof_f1:.4f}")

final_test_preds_avg = np.mean(all_test_preds, axis=0)
test_df['label'] = (final_test_preds_avg[:, 1] > 0.465).astype(int)

submission_df = test_df[['ID', 'label']]
submission_output_path = os.path.join(CFG.PARENT_SAVE_PATH, "submission.csv")
submission_df.to_csv(submission_output_path, index=False)
print(f"Submission file saved to: {submission_output_path}")